In [ ]:
import math 
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite
import networkx as nx
from ortools.linear_solver import pywraplp as OR
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import copy
import pickle
from bokeh import palettes
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import (GraphRenderer, Circle, MultiLine, StaticLayoutProvider,
                          HoverTool, TapTool, EdgesAndLinkedNodes, NodesAndLinkedEdges,
                          ColumnDataSource, LabelSet, NodesOnly)
from bipartite_matching import *
output_notebook()

## Retrospective Minimum Fleet-size Problem to Bipartite Matching Formulation

Consider a minimum fleet-size problem over some time horizon. Over this time period, we know all the ride requests we will receive. Each ride request has a start/end location, start time, and duration. A retrospective management question is to find the minimum number of vehicles needed to cover all the rides. How can we create a bipartite matching formulation to solve this problem?

In [ ]:
# Load the taxi trips information as well as NYC street nodes and arcs.
trips_df = pd.read_csv('data/2013-09-01_trip_data_manhattan.csv').drop(columns='id')
nodes_df = pd.read_csv('data/nyc_nodes_manhattan.csv').drop(columns='Unnamed: 0')
arcs_df = pd.read_csv('data/nyc_links_manhattan.csv').drop(columns='Unnamed: 0')
times_df = pd.read_csv('data/times.csv', index_col =0)

# Convert the travel time matrix to a dictionary
times = dict()
for label, content in times_df.items():
    for idx in content.index:
        times[int(idx), int(label)] = content[idx]

In [ ]:
# A list of 10 example trip_ids
ex_trips = [68894, 69893, 68634, 69166, 69572, 66416, 69963, 71686, 68768, 68536]
# Locate the corresponding trip information
trips = trips_df.iloc[ex_trips]
trips

In [ ]:
# Intialize nodes and edges
DO_nodes = list()
PU_nodes = list()
edges = list()
# Initialize a dict that maps a PU node to a DO node
PUtoDO = dict()
# Construct nodes and edges
for index, row in trips.iterrows():
    s = row['start_node']
    t = row['end_node']
    s_t = row['start_time']
    t_t = s_t + row['trip_time']
    DO_node = (int(t), t_t, index, 'DO')
    PU_node = (int(s), s_t, index, 'PU')
    DO_nodes.append(DO_node)
    PU_nodes.append(PU_node)
    PUtoDO[PU_node] = DO_node
# Sort the nodes by time
DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
PU_nodes = sorted(PU_nodes, key = lambda x: x[1])

# Specify the edges
max_waiting_time = 10 # delta

for DO_node in DO_nodes:
    for PU_node in PU_nodes:
        if PU_node[1] > DO_node[1] + max_waiting_time:
            break
        else:
            if PU_node[1] >= DO_node[1]:
                time = times[(DO_node[0], PU_node[0])]
                if ((PU_node[1] - DO_node[1]) - max_waiting_time  <= time) & (time <= (PU_node[1] - DO_node[1])):
                    edges.append((DO_node, PU_node))

In [ ]:
# Generate the bipartite graph
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(DO_nodes, bipartite=0)
B.add_nodes_from(PU_nodes, bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_edges_from(edges)

In [ ]:
match = nx.bipartite.maximum_matching(B, DO_nodes)
print('Size of max cardinality matching:', int(len(match)/2)) # divided by two because the output edges are directed

In [ ]:
# Trace optimal taxi trajetory
opt_paths = match_to_path(match, trips)

In [ ]:
# Plot the bipartite graph
plot_ex_bipartite(B, match, opt_paths, True)

Only 4 taxis are needed to serve all 10 trips according to our matching obtained above.

In [ ]:
# Plot the corresponding taxi paths on the map
G = street_network(nodes_df, arcs_df)
plot_taxi_route(G, opt_paths, nodes_df)

## Retrospective Minimum Fleet-size Problem At Scale

Now, filter the trips by time window of interest. The following example selects all the trips from 5 pm to 5:30 pm.

In [ ]:
# Filter trips by time window of interest
start_time = 1020
end_time = 1050
trips = trips_df.copy()
trips = trips[(trips.start_time >= start_time) & 
              (trips.start_time + trips.trip_time <= end_time)].copy()
trips.start_time = trips.start_time - start_time

In [ ]:
# Intialize nodes and edges
DO_nodes = list()
PU_nodes = list()
edges = list()
# Initialize a dict that maps a PU node to a DO node
PUtoDO = dict()
# Specify nodes - each node is a tuple of (location_id, time, trip_id, "DO"/"PU")
for index, row in trips.iterrows():
    s = row['start_node']
    t = row['end_node']
    s_t = row['start_time']
    t_t = s_t + row['trip_time']
    DO_node = (int(t), t_t, index, 'DO')
    PU_node = (int(s), s_t, index, 'PU')
    DO_nodes.append(DO_node)
    PU_nodes.append(PU_node)
    PUtoDO[PU_node] = DO_node
# Sort the nodes by time
DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
PU_nodes = sorted(PU_nodes, key = lambda x: x[1])

# Specify edges
max_waiting_time = 10

for DO_node in DO_nodes:
    for PU_node in PU_nodes:
        if PU_node[1] > DO_node[1] + max_waiting_time:
            break
        else:
            if PU_node[1] >= DO_node[1]:
                time = times[(DO_node[0], PU_node[0])]
                if ((PU_node[1] - DO_node[1]) - max_waiting_time  <= time) & (time <= (PU_node[1] - DO_node[1])):
                    edges.append((DO_node, PU_node))
# load the model
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(DO_nodes, bipartite=0)
B.add_nodes_from(PU_nodes, bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_edges_from(edges)

In [ ]:
match = nx.bipartite.maximum_matching(B, DO_nodes)
print('size of max cardinality matching:', len(match) / 2)
print('total number of trips:', len(DO_nodes))
num_taxi = len(DO_nodes) - len(match)/2
print('min number of taxis needed to cover all trips:', num_taxi)

In [ ]:
# Trace optimal taxi trajetory
opt_paths = match_to_path(match, trips)
# Plot the first 10 taxi paths on the map
G = street_network(nodes_df, arcs_df)
plot_taxi_route(G, opt_paths[:10], nodes_df)